In [7]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import math
import plotly.graph_objects as go

In [8]:
#Self Explanatory
start = dt.datetime(2010,5,1)
end = dt.datetime(2021,5,10)

stock = "SPY"

s_df = web.DataReader(stock,'yahoo',start,end)

s_df['Adj Low'] = (s_df['Low']*s_df['Adj Close'])/s_df['Close']
s_df['Adj High'] = (s_df['High']*s_df['Adj Close'])/s_df['Close']
s_df['Adj Open'] = (s_df['Open']*s_df['Adj Close'])/s_df['Close']
s_df = s_df.drop(['Volume','Open','High','Low','Close'],axis=1)
s_df.head()

,Adj Close,Adj Low,Adj High,Adj Open
Date,,,,
2010-05-03,96.810913,95.885839,97.076370,96.030633
2010-05-04,94.534447,94.051801,95.749110,95.733024
2010-05-05,93.971329,93.287580,94.759655,93.762180
2010-05-06,90.850220,84.463192,94.116128,93.520865
2010-05-07,89.498802,88.010643,91.517870,90.608888


In [9]:
#Also kinda self explanatory
r_df = pd.DataFrame(index = s_df.index)

r_df['Absolute returns'] = s_df['Adj Close'].pct_change()
r_df['Log returns'] = np.log(1+r_df['Absolute returns'])

stdev = r_df['Log returns'].dropna().std()
mean = r_df['Log returns'].dropna().mean()


In [10]:
#Upper and lower Taleb bands according to https://prnt.sc/12fljew
lowerTalebBand = mean-stdev*math.sqrt(5-math.sqrt(17)/2)
upperTalebBand = mean+stdev*math.sqrt(5-math.sqrt(17)/2)
r_df['lowerTalebBand'] = lowerTalebBand
r_df['upperTalebBand'] = upperTalebBand

#Upper and Lower 30 period, 2 stdev Bollinger Bands
r_df["MA"] = r_df['Log returns'].rolling(window=30).mean()
r_df["MSTD"] = r_df['Log returns'].rolling(window=30).std()
r_df["Upper Bollinger"] = r_df['MA']+2*r_df["MSTD"]
r_df["Lower Bollinger"] = r_df['MA']-2*r_df["MSTD"]

In [11]:
#Calculates time passed since a return crossed one of the bands (Taleb or Bollinger)

r_df['lastCrossTaleb'] = 0
r_df['lastCrossMoving'] = 0

for i in range(len(r_df.index)):

    ret = r_df.iloc[i]['Log returns']
    if(ret<upperTalebBand and ret>lowerTalebBand):
        r_df.loc[r_df.index[i],'lastCrossTaleb']= 1+r_df.iloc[i-1]['lastCrossTaleb']

    lower = r_df.iloc[i]['Lower Bollinger']
    upper = r_df.iloc[i]['Upper Bollinger']

    if(ret<upper and ret>lower):
        r_df.loc[r_df.index[i],'lastCrossMoving']= r_df.iloc[i-1]['lastCrossMoving']+1
#I know about the warning but fixing it is way too complicated



In [12]:

#Graphing stuff

candlesticks = go.Candlestick(x=s_df.index,
                open=s_df['Adj Open'],
                high=s_df['Adj High'],
                low=s_df['Adj Low'],
                close=s_df['Adj Close'],name = "OHLC")

lowerTalebBandChart = go.Scatter(x=r_df.index, y=100*r_df['lowerTalebBand'],
 mode='lines', name='Lower Taleb Band',xaxis="x",yaxis="y2")
upperTalebBandChart = go.Scatter(x=r_df.index, y=100*r_df['upperTalebBand'],
 mode='lines', name='Upper Taleb Band',xaxis="x",yaxis="y2")

returnsChart = go.Scatter(x=r_df.index, y=100*r_df['Log returns'],
 mode='lines', name='Returns',xaxis="x",yaxis="y2")

lowerBollingerChart = go.Scatter(x=r_df.index, y=100*r_df['Lower Bollinger'],
 mode='lines', name='Lower Bollinger',xaxis="x",yaxis="y2")
upperBollingerChart = go.Scatter(x=r_df.index, y=100*r_df['Upper Bollinger'], 
mode='lines', name='Upper Bollinger',xaxis="x",yaxis="y2")

crossTalebChart = go.Scatter(x=r_df.index, y=r_df['lastCrossTaleb'],
 mode='lines', name='Last Cross Taleb',xaxis="x",yaxis="y3")
crossBollingerChart = go.Scatter(x=r_df.index, y=r_df['lastCrossMoving'],
 mode='lines', name='Last Cross Bollinger',xaxis="x",yaxis="y3")

bollingerDifChart = go.Scatter(x=r_df.index, y=r_df['Upper Bollinger']-r_df['Lower Bollinger'],
 mode='lines', name='Bollinger Difference',xaxis="x",yaxis="y3")
talebDifChart = go.Scatter(x=r_df.index, y=r_df['upperTalebBand']-r_df['lowerTalebBand'],
 mode='lines', name='Taleb Difference',xaxis="x",yaxis="y3")
    

layout = go.Layout(
    height=800,
    width=1100,
    title=stock,
    xaxis=dict(
        domain=[0, 1],
    ),
    yaxis=dict(
        domain=[0.55, 1]
    ),
    yaxis2=dict(
        domain=[0.20, 0.50]
    ),
    yaxis3=dict(
        domain=[0, 0.15]
    )
)


#To change the charts just add or remove them from this list
fig = go.Figure(data=[candlesticks,
lowerTalebBandChart,
upperTalebBandChart,
returnsChart,
lowerBollingerChart,
upperBollingerChart,
bollingerDifChart,
talebDifChart],
layout=layout)

fig.update_layout(xaxis_rangeslider_visible=False)


fig.show()
